In [ ]:
# 🏥 HealthBot: AI-Powered Patient Education System

## 📦 Installation
Run this command in your terminal **before** running the notebook:
```bash
pip install -r requirements.txt
```

## 🔑 Setup Instructions
1. Create a file named `config.env` in the project folder
2. Add your API keys to `config.env`:
   ```
   OPENAI_API_KEY="voc-your-vocareum-key-here"
   TAVILY_API_KEY="tvly-your-tavily-key-here"
   ```
3. Get a free Tavily API key at: https://tavily.com/

## 🐛 Troubleshooting

### Issue: "Search results are garbage/random characters"
**Cause:** Tavily API key may be invalid or expired
**Solution:**
1. Verify your Tavily API key at https://tavily.com/
2. Make sure the key in `config.env` is correct
3. Try regenerating your Tavily API key
4. Run Cell 3 (Test Cell) to debug the search

### Issue: "SSL Certificate Errors"
**Cause:** Vocareum environment certificate verification
**Solution:** The notebook includes automatic SSL handling - just run it!

### Issue: "No module named 'langchain'"
**Cause:** Dependencies not installed
**Solution:** Run `pip install -r requirements.txt` in terminal

## 📖 How to Use
1. Run Cell 1: Load API Keys
2. Run Cell 2: Initialize HealthBot
3. (Optional) Run Cell 3: Test Search
4. Follow the interactive prompts!

In [ ]:
# Load in the OpenAI key and Tavily key.
# 
# 📝 SETUP INSTRUCTIONS:
# 1. Create a file named 'config.env' in the project folder
# 2. Add the following lines to config.env:
#    OPENAI_API_KEY="voc-your-vocareum-key-here"
#    TAVILY_API_KEY="tvly-your-tavily-key-here"
#
# 🔑 Getting API Keys:
# - Vocareum OpenAI key: Provided by your course/instructor
# - Tavily key: Sign up at https://tavily.com/ (free tier available)
#
# ⚠️ Important: Keep your API keys secure and never share them publicly!

from dotenv import load_dotenv
import os 

# Load environment variables
load_dotenv('config.env')

# Verify API keys are loaded
try:
    assert os.getenv('OPENAI_API_KEY') is not None, "OPENAI_API_KEY not found in config.env"
    assert os.getenv('TAVILY_API_KEY') is not None, "TAVILY_API_KEY not found in config.env"
    print("✅ API keys loaded successfully!")
    print(f"   OpenAI Key: {os.getenv('OPENAI_API_KEY')[:10]}...")
    print(f"   Tavily Key: {os.getenv('TAVILY_API_KEY')[:10]}...")
except AssertionError as e:
    print(f"❌ Error: {e}")
    print("\n📝 Please create a 'config.env' file with your API keys.")
    print("   See the instructions above for details.")
    raise

In [7]:
import time 
import os
import ssl
from typing import Literal
from langgraph.graph import MessagesState, START, StateGraph, END
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.runnables import RunnableConfig
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_community.tools.tavily_search import TavilySearchResults
from IPython.display import Image, display

# Workaround for SSL certificate verification issues in Vocareum environment
# This allows connections to the Vocareum OpenAI proxy
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

# Helper functions for better formatted output
def print_header(title, icon="🏥"):
    """Print a nicely formatted header"""
    line = "═" * 70
    print(f"\n{line}")
    print(f"{icon}  {title.upper()}")
    print(f"{line}\n")
    time.sleep(0.3)

def print_section(title, icon="📋"):
    """Print a section header"""
    print(f"\n{icon} {title}")
    print("─" * 70)
    time.sleep(0.2)

def print_info(text, indent=2):
    """Print formatted information with indentation"""
    spaces = " " * indent
    # Handle multi-line text
    lines = text.split('\n')
    for line in lines:
        print(f"{spaces}{line}")
    time.sleep(0.2)

def print_success(text, icon="✅"):
    """Print success message"""
    print(f"\n{icon} {text}\n")
    time.sleep(0.2)

def print_warning(text, icon="⚠️"):
    """Print warning message"""
    print(f"\n{icon} {text}\n")
    time.sleep(0.2)

def print_step(step_num, total_steps, description):
    """Print step progress"""
    print(f"\n🔹 Step {step_num}/{total_steps}: {description}")
    time.sleep(0.2)

def display_text_to_user(text):
    """Legacy function for compatibility"""
    print(text) 
    time.sleep(0.3)
    
def ask_user_for_input(input_description, icon="💬"):
    """Get user input with nice formatting"""
    print(f"\n{icon} {input_description}")
    response = input("   ➤ ")
    return response

# Define State class with all necessary properties
class State(MessagesState):
    topic: str  # The health topic the patient wants to learn about
    search_results: str  # Raw search results from Tavily
    summary: str  # Patient-friendly summary
    quiz_question: str  # Generated quiz question
    user_answer: str  # Patient's answer to the quiz
    grade: str  # Grade for the answer (A, B, C, etc.)
    explanation: str  # Explanation of the grade
    continue_learning: str  # Whether to continue or exit

# Initialize model and tools with Vocareum configuration
# Using httpx to handle SSL certificate verification for Vocareum proxy
import httpx
import warnings

# Suppress SSL warnings for cleaner output in Vocareum environment
warnings.filterwarnings('ignore', message='Unverified HTTPS request')

# Create an HTTP client that doesn't verify SSL certificates (for Vocareum)
http_client = httpx.Client(verify=False)

model = ChatOpenAI(
    temperature=0, 
    streaming=False, 
    model="gpt-4",
    base_url="https://openai.vocareum.com/v1",
    api_key=os.getenv('OPENAI_API_KEY'),
    http_client=http_client
)
tavily_tool = TavilySearchResults(max_results=3)

# Create workflow
workflow = StateGraph(State)

# Node 1: Collect topic from patient
def collect_topic(state):
    print_header("Welcome to HealthBot", "🏥")
    print_info("Your AI-Powered Health Education Assistant")
    print_info("━" * 70, indent=0)
    
    print_info("\n📚 What you'll learn:")
    print_info("  • Clear, easy-to-understand health information", indent=4)
    print_info("  • Key facts about medical conditions", indent=4)
    print_info("  • Treatment options and management strategies", indent=4)
    print_info("  • A quiz to test your understanding\n", indent=4)
    
    print_info("💡 Instructions:")
    print_info("  • Enter any health topic or medical condition", indent=4)
    print_info("  • Examples: 'diabetes', 'high blood pressure', 'vitamin D deficiency'", indent=4)
    print_info("  • Be specific for better results\n", indent=4)
    
    topic = ask_user_for_input("What health topic or medical condition would you like to learn about?", "🔍")
    
    if topic.strip():
        print_success(f"Excellent choice! Searching for information about: '{topic}'", "🎯")
    
    return {"topic": topic}

# Node 2: Search using Tavily
def search_topic(state):
    print_step(1, 4, f"Searching for reliable medical information about '{state['topic']}'")
    print_info("🔎 Scanning medical databases and trusted health sources...")
    print_info("📡 Retrieving the latest information...", indent=2)
    
    # Use Tavily to search for the topic
    search_query = f"medical information about {state['topic']} symptoms causes treatment"
    
    try:
        results = tavily_tool.invoke({"query": search_query})
        
        # Format results into a string
        search_results_text = ""
        source_count = 0
        
        # Tavily returns a list of documents/dictionaries
        if isinstance(results, list):
            for i, result in enumerate(results):
                if isinstance(result, dict):
                    # Extract content and url from dictionary
                    content = result.get('content', result.get('snippet', ''))
                    url = result.get('url', 'N/A')
                    
                    if content and content.strip():
                        source_count += 1
                        search_results_text += f"\\n=== Source {source_count} ===\\n"
                        search_results_text += f"URL: {url}\\n"
                        search_results_text += f"Content:\\n{content}\\n"
                        search_results_text += "=" * 50 + "\\n"
                
                # Handle Document-like objects with page_content attribute
                elif hasattr(result, 'page_content'):
                    content = result.page_content
                    metadata = getattr(result, 'metadata', {})
                    url = metadata.get('source', metadata.get('url', 'N/A'))
                    
                    if content and content.strip():
                        source_count += 1
                        search_results_text += f"\\n=== Source {source_count} ===\\n"
                        search_results_text += f"URL: {url}\\n"
                        search_results_text += f"Content:\\n{content}\\n"
                        search_results_text += "=" * 50 + "\\n"
        
        # If no valid results found
        if source_count == 0 or not search_results_text.strip():
            print_warning("Limited results found. Using basic search...", "⚠️")
            search_results_text = f"Topic: {state['topic']}\\n\\nPlease provide a general overview of {state['topic']} including its definition, common symptoms or characteristics, typical causes or risk factors, and available treatment or management options."
            source_count = 1
        
        print_success(f"Found {source_count} reliable source(s)! Preparing your summary...", "✅")
        
        return {"search_results": search_results_text}
        
    except Exception as e:
        print_warning(f"Search encountered an issue: {str(e)[:100]}", "⚠️")
        # Provide fallback
        search_results_text = f"Topic: {state['topic']}\\n\\nPlease provide a comprehensive overview of {state['topic']} based on general medical knowledge."
        return {"search_results": search_results_text}

# Node 3: Summarize search results
def summarize_results(state):
    print_step(2, 4, "Creating your personalized health summary")
    print_info("🤖 AI is analyzing the information...")
    print_info("📝 Writing in patient-friendly language...", indent=2)
    print_info("⏳ This may take a moment...", indent=2)
    
    # Create prompt for summarization
    prompt = f"""You are a medical educator. Based ONLY on the following search results, create a patient-friendly summary about {state['topic']}.

Search Results:
{state['search_results']}

Create a 3-4 paragraph summary that:
1. Explains what {state['topic']} is in simple terms
2. Describes key symptoms, causes, or characteristics
3. Discusses treatment options or management strategies
4. Provides relevant health information for patients

Use ONLY the information from the search results above. Write in a clear, compassionate, and accessible way for patients.

Summary:"""

    messages = [
        SystemMessage(content="You are a medical education assistant helping patients understand health topics."),
        HumanMessage(content=prompt)
    ]
    
    response = model.invoke(messages)
    summary = response.content
    
    # Add to messages for context
    new_messages = state['messages'] + messages + [response]
    
    return {"summary": summary, "messages": new_messages}

# Node 4: Display summary and wait for user
def display_summary(state):
    print_section(f"📚 Health Information Summary: {state['topic'].title()}", "📋")
    print_info("\n" + "─" * 70, indent=0)
    
    # Display the summary with nice formatting
    summary_lines = state['summary'].split('\n')
    for line in summary_lines:
        if line.strip():
            print_info(line, indent=4)
    
    print_info("\n" + "─" * 70, indent=0)
    print_success("Summary complete! Please take your time to read and understand the information above.", "📖")
    
    print_info("\n💡 Next Step:")
    print_info("  • Review the summary carefully", indent=4)
    print_info("  • When you're ready, we'll test your understanding with a quiz", indent=4)
    print_info("  • Type 'ready' when you want to continue\n", indent=4)
    
    ready = ask_user_for_input("Are you ready for the comprehension check? (type 'ready' to continue)", "✍️")
    
    if ready.strip().lower() == 'ready':
        print_success("Great! Let's test your knowledge!", "🎓")
    
    # LangGraph requires updating at least one state field, so we pass through existing state
    return {"summary": state['summary']}

# Node 5: Generate quiz question
def generate_quiz(state):
    print_step(3, 4, "Generating a quiz to test your understanding")
    print_info("🧠 Creating a custom question based on the summary...")
    print_info("📝 This will help reinforce your learning...", indent=2)
    
    prompt = f"""Based ONLY on the following summary about {state['topic']}, create ONE multiple-choice quiz question to test patient comprehension.

Summary:
{state['summary']}

Create a question that:
1. Tests understanding of key concepts from the summary
2. Has 4 answer options (A, B, C, D)
3. Is answerable using ONLY information from the summary above
4. Is appropriate for a patient audience

Format your response exactly as:
Question: [Your question here]
A) [Option A]
B) [Option B]
C) [Option C]
D) [Option D]
Correct Answer: [Letter]

Quiz:"""

    messages = [
        SystemMessage(content="You are a medical educator creating quiz questions for patient education."),
        HumanMessage(content=prompt)
    ]
    
    response = model.invoke(messages)
    quiz_question = response.content
    
    # Add to messages for context
    new_messages = state['messages'] + messages + [response]
    
    return {"quiz_question": quiz_question, "messages": new_messages}

# Node 6: Display quiz and collect answer
def collect_answer(state):
    print_section("📝 Comprehension Check Quiz", "🎯")
    print_info("\n💡 Instructions:")
    print_info("  • Read the question carefully", indent=4)
    print_info("  • Choose the best answer from options A, B, C, or D", indent=4)
    print_info("  • Type only the letter of your answer\n", indent=4)
    
    print_info("─" * 70, indent=0)
    
    # Display quiz question with nice formatting
    quiz_lines = state['quiz_question'].split('\n')
    for line in quiz_lines:
        if line.strip():
            if line.startswith('Question:'):
                print_info(f"\n❓ {line}", indent=2)
            elif line.strip().startswith(('A)', 'B)', 'C)', 'D)')):
                print_info(f"   {line}", indent=4)
            elif not line.startswith('Correct Answer:'):
                print_info(line, indent=4)
    
    print_info("\n" + "─" * 70, indent=0)
    
    user_answer = ask_user_for_input("Your answer (enter A, B, C, or D)", "✍️")
    answer = user_answer.strip().upper()
    
    if answer in ['A', 'B', 'C', 'D']:
        print_success(f"Answer recorded: {answer}", "✅")
    else:
        print_warning("Please note: Valid answers are A, B, C, or D", "⚠️")
    
    return {"user_answer": answer}

# Node 7: Grade the answer
def grade_answer(state):
    print_step(4, 4, "Evaluating your answer")
    print_info("🤖 AI is reviewing your response...")
    print_info("📊 Preparing detailed feedback...", indent=2)
    
    prompt = f"""You are a medical educator grading a patient's quiz answer.

Summary:
{state['summary']}

Quiz Question:
{state['quiz_question']}

Patient's Answer: {state['user_answer']}

Grade the patient's answer and provide feedback:
1. Assign a letter grade (A for correct, F for incorrect, or partial credit B/C/D if partially correct or shows understanding)
2. Explain why the answer received that grade
3. Include relevant citations from the summary to reinforce learning
4. Be encouraging and educational

Format your response as:
Grade: [Letter Grade]
Explanation: [Your detailed explanation with citations from the summary]

Grading:"""

    messages = [
        SystemMessage(content="You are a compassionate medical educator providing constructive feedback."),
        HumanMessage(content=prompt)
    ]
    
    response = model.invoke(messages)
    grading_response = response.content
    
    # Parse grade and explanation
    lines = grading_response.split('\\n', 1)
    grade = lines[0].replace('Grade:', '').strip() if 'Grade:' in lines[0] else 'N/A'
    explanation = lines[1].replace('Explanation:', '').strip() if len(lines) > 1 else grading_response
    
    # Add to messages for context
    new_messages = state['messages'] + messages + [response]
    
    return {"grade": grade, "explanation": explanation, "messages": new_messages}

# Node 8: Display grade and ask to continue
def display_grade_and_ask_continue(state):
    print_section("📊 Your Quiz Results", "🎓")
    
    # Display grade with appropriate icon
    grade = state['grade'].strip()
    grade_icon = "🌟" if grade in ['A', 'A+', 'A-'] else "✅" if grade in ['B', 'B+', 'B-'] else "📝"
    
    print_info(f"\n{grade_icon} Your Grade: {grade}", indent=2)
    print_info("─" * 70, indent=0)
    
    print_info("\n💬 Detailed Feedback:", indent=2)
    explanation_lines = state['explanation'].split('\n')
    for line in explanation_lines:
        if line.strip():
            print_info(line, indent=4)
    
    print_info("\n" + "─" * 70, indent=0)
    
    # Encouragement message based on grade
    if grade in ['A', 'A+', 'A-']:
        print_success("Excellent work! You have a great understanding of this topic! 🌟", "🎉")
    elif grade in ['B', 'B+', 'B-', 'C', 'C+', 'C-']:
        print_success("Good effort! Review the feedback to strengthen your understanding. 📚", "👍")
    else:
        print_info("\n  💡 Tip: Review the summary again to better understand the material.", indent=2)
    
    print_info("\n" + "═" * 70, indent=0)
    print_info("\n🔄 What would you like to do next?", indent=2)
    print_info("\n  [1] 📚 Learn about another health topic", indent=4)
    print_info("  [2] 🚪 Exit HealthBot\n", indent=4)
    
    choice = ask_user_for_input("Enter your choice (1 or 2)", "🔢")
    
    if choice.strip() == '1':
        print_success("Great! Let's explore another health topic!", "🎯")
    elif choice.strip() == '2':
        print_info("\n  Thank you for using HealthBot! Stay healthy! 👋", indent=2)
    
    return {"continue_learning": choice.strip()}

# Node 9: Reset state for new topic
def reset_for_new_topic(state):
    print_info("\n" + "═" * 70, indent=0)
    print_success("Preparing a new learning session...", "🔄")
    print_info("  • Previous session data cleared for privacy", indent=4)
    print_info("  • Ready for a new health topic\n", indent=4)
    time.sleep(0.5)
    
    # Reset all topic-specific state but keep messages empty for privacy
    return {
        "messages": [],
        "topic": "",
        "search_results": "",
        "summary": "",
        "quiz_question": "",
        "user_answer": "",
        "grade": "",
        "explanation": "",
        "continue_learning": ""
    }

# Node 10: End session
def end_session(state):
    print_info("\n" + "═" * 70, indent=0)
    print_header("Thank You for Using HealthBot!", "🏥")
    
    print_info("📚 Remember:")
    print_info("  • Stay informed about your health", indent=4)
    print_info("  • Always consult healthcare professionals for medical advice", indent=4)
    print_info("  • Keep learning and stay healthy!\n", indent=4)
    
    print_success("Session ended. Take care! 👋", "✨")
    print_info("═" * 70 + "\n", indent=0)
    
    # LangGraph requires updating at least one state field
    return {"continue_learning": "2"}

# Conditional edge function
def should_continue(state) -> Literal["reset_for_new_topic", "end_session"]:
    if state.get('continue_learning') == '1':
        return "reset_for_new_topic"
    else:
        return "end_session"

# Add nodes to workflow
workflow.add_node("collect_topic", collect_topic)
workflow.add_node("search_topic", search_topic)
workflow.add_node("summarize_results", summarize_results)
workflow.add_node("display_summary", display_summary)
workflow.add_node("generate_quiz", generate_quiz)
workflow.add_node("collect_answer", collect_answer)
workflow.add_node("grade_answer", grade_answer)
workflow.add_node("display_grade_and_ask_continue", display_grade_and_ask_continue)
workflow.add_node("reset_for_new_topic", reset_for_new_topic)
workflow.add_node("end_session", end_session)

# Add edges to workflow
workflow.add_edge(START, "collect_topic")
workflow.add_edge("collect_topic", "search_topic")
workflow.add_edge("search_topic", "summarize_results")
workflow.add_edge("summarize_results", "display_summary")
workflow.add_edge("display_summary", "generate_quiz")
workflow.add_edge("generate_quiz", "collect_answer")
workflow.add_edge("collect_answer", "grade_answer")
workflow.add_edge("grade_answer", "display_grade_and_ask_continue")

# Conditional edge: continue or exit
workflow.add_conditional_edges(
    "display_grade_and_ask_continue",
    should_continue,
    {
        "reset_for_new_topic": "reset_for_new_topic",
        "end_session": "end_session"
    }
)

# Connect reset back to collect_topic to restart the flow
workflow.add_edge("reset_for_new_topic", "collect_topic")
workflow.add_edge("end_session", END)

# Compile workflow
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

# Display workflow graph (optional - may fail due to SSL issues)
print("\n" + "═" * 70)
print("🔧 INITIALIZING HEALTHBOT WORKFLOW")
print("═" * 70)

try:
    print("\n📊 Workflow graph visualization:")
    display(Image(app.get_graph().draw_mermaid_png()))
    print("✅ Workflow compiled successfully!\n")
except Exception as e:
    print(f"\n⚠️  Note: Could not render workflow graph visualization")
    print(f"   Reason: {str(e)[:80]}...")
    print("   ✅ The workflow will still run correctly without the visualization.\n")

# Run the HealthBot
config = RunnableConfig(recursion_limit=2000, configurable={"thread_id": "healthbot_session"})  

initial_state = {
    "messages": [],
    "topic": "",
    "search_results": "",
    "summary": "",
    "quiz_question": "",
    "user_answer": "",
    "grade": "",
    "explanation": "",
    "continue_learning": ""
}

print("\n" + "═" * 70)
print("🚀 STARTING HEALTHBOT SESSION")
print("═" * 70)
print("\n💡 System Status:")
print("   ✅ AI Model: Connected (GPT-4 via Vocareum)")
print("   ✅ Search Engine: Ready (Tavily)")
print("   ✅ Knowledge Base: Active")
print("   ✅ Quiz Generator: Initialized")
print("\n" + "═" * 70 + "\n")
time.sleep(0.5)
    
app.invoke(initial_state, config)


══════════════════════════════════════════════════════════════════════
🔧 INITIALIZING HEALTHBOT WORKFLOW
══════════════════════════════════════════════════════════════════════

📊 Workflow graph visualization:

⚠️  Note: Could not render workflow graph visualization
   Reason: Failed to reach https://mermaid.ink API while trying to render your graph after ...
   ✅ The workflow will still run correctly without the visualization.


══════════════════════════════════════════════════════════════════════
🚀 STARTING HEALTHBOT SESSION
══════════════════════════════════════════════════════════════════════

💡 System Status:
   ✅ AI Model: Connected (GPT-4 via Vocareum)
   ✅ Search Engine: Ready (Tavily)
   ✅ Knowledge Base: Active
   ✅ Quiz Generator: Initialized

══════════════════════════════════════════════════════════════════════


══════════════════════════════════════════════════════════════════════
🏥  WELCOME TO HEALTHBOT
══════════════════════════════════════════════════════════════════

{'messages': [SystemMessage(content='You are a medical education assistant helping patients understand health topics.', additional_kwargs={}, response_metadata={}, id='cd92d3a8-6da5-468f-924e-4fc08d444f98'),
  HumanMessage(content='You are a medical educator. Based ONLY on the following search results, create a patient-friendly summary about high blood pressure.\n\nSearch Results:\nTopic: high blood pressure\\n\\nPlease provide a general overview of high blood pressure including its definition, common symptoms or characteristics, typical causes or risk factors, and available treatment or management options.\n\nCreate a 3-4 paragraph summary that:\n1. Explains what high blood pressure is in simple terms\n2. Describes key symptoms, causes, or characteristics\n3. Discusses treatment options or management strategies\n4. Provides relevant health information for patients\n\nUse ONLY the information from the search results above. Write in a clear, compassionate, and accessible way for patient

In [ ]:
# OPTIONAL: Test Tavily Search (Run this cell to debug search issues)
# Uncomment and run this cell if you want to test the Tavily search independently

"""
print("🔍 Testing Tavily Search...")
test_query = "diabetes symptoms and treatment"
print(f"Query: {test_query}\n")

try:
    test_results = tavily_tool.invoke({"query": test_query})
    print(f"✅ Search successful!")
    print(f"Result type: {type(test_results)}")
    print(f"Number of results: {len(test_results) if isinstance(test_results, list) else 'N/A'}")
    
    if isinstance(test_results, list) and len(test_results) > 0:
        print(f"\nFirst result structure:")
        first_result = test_results[0]
        print(f"  Type: {type(first_result)}")
        
        if isinstance(first_result, dict):
            print(f"  Keys: {list(first_result.keys())}")
            print(f"  Sample content: {str(first_result)[:200]}...")
        elif hasattr(first_result, 'page_content'):
            print(f"  Has page_content: True")
            print(f"  Content preview: {first_result.page_content[:200]}...")
            
except Exception as e:
    print(f"❌ Search failed: {e}")
    print(f"Error type: {type(e)}")
"""
